## Add Operators

In [ ]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import json

from src.lm_store import LMStore
from src.llm_adapter import chat_model, embedding_model

In [ ]:
load_dotenv()

chat = chat_model(    
        provider="azureopenai",
        model=os.getenv("CHAT_MODEL"),
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        key=os.getenv("AZURE_OPENAI_KEY"),
        api_version=os.getenv("AZURE_OPENAI_VERSION")
        )

embed = embedding_model(    
        provider="azureopenai",
        model=os.getenv("EMBEDDING_MODEL"),
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        key=os.getenv("AZURE_OPENAI_KEY"),
        api_version=os.getenv("AZURE_OPENAI_VERSION")
        )
# Initialize LMStore
lm_store = LMStore(chat, embed)

# Load tools from JSON file
with open("data/tools.json", "r", encoding="utf-8") as f:
    tools = json.load(f)

# Add all tools to LMStore
added_count = lm_store.add(tools)

print(f"Added {added_count} tools to LMStore")
print(f"Total tools in store: {lm_store.count}")

lm_store.export("data/lmstore.db")

## Search Operator

In [ ]:
load_dotenv()

chat = chat_model(    
        provider="ollama",
        model=os.getenv("CHAT_MODEL"),
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        key=os.getenv("AZURE_OPENAI_KEY"),
        api_version=os.getenv("AZURE_OPENAI_VERSION")
        )

embed = embedding_model(    
        provider="azureopenai",
        model=os.getenv("EMBEDDING_MODEL"),
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        key=os.getenv("AZURE_OPENAI_KEY"),
        api_version=os.getenv("AZURE_OPENAI_VERSION")
        )

lm_store = LMStore(chat, embed)

lm_store.load_from_file('data/lmstore.db')

query = "Create a new branch in a GitHub repository"    

results, selection = lm_store.get(
    query=query,
    limit=5,
    llm_search=True,
)

print("\nSearch results:")
for r in results:
    print(f"  {r.name}: {r.similarity_score:.4f}")

if selection:
    print(f"\nLLM selected: {selection}")